# **Merge Train Meteo + Google Trends**

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot') 

import warnings 
warnings.filterwarnings ('ignore')

In [6]:
train_meteo = pd.read_csv("../data/processed/train_meteo.csv")
train_google = pd.read_csv("../data/processed/train_final.csv")

print("Train meteo shape:", train_meteo.shape)
print("Train google shape:", train_google.shape)

Train meteo shape: (9196, 14)
Train google shape: (9196, 24)


In [7]:
train_meteo.head()

,Id,week,region_code,region_name,TauxGrippe,year,week_num,date,month,t,pmer,ff,rr24,u
0,5523,201152,42,ALSACE,66,2011,52,2011-12-26,12,278.613306,101860.544355,3.887702,179.4,83.187500
1,5524,201152,72,AQUITAINE,24,2011,52,2011-12-26,12,282.006707,102447.682927,3.197154,305.3,87.701220
2,5525,201152,83,AUVERGNE,91,2011,52,2011-12-26,12,278.906452,102179.798387,3.413911,106.7,80.453629
3,5526,201152,25,BASSE-NORMANDIE,49,2011,52,2011-12-26,12,281.327209,101840.124031,6.089130,336.3,85.221705
4,5527,201152,26,BOURGOGNE,33,2011,52,2011-12-26,12,278.612857,102035.428571,4.084898,107.2,88.216327


In [8]:
train_google.head()

,Id,week,region_code,region_name,TauxGrippe,year,Mois_x,month,region_normalized,region,...,pop_75_plus,pop_total,Mois_y,requete_grippe_x,requete_grippe_aviaire_vaccin_x,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_x,Mois,requete_grippe_y,requete_grippe_aviaire_vaccin_y,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_y
0,5523,201152,42,ALSACE,66,2011,12,2011-12-01,ALSACE,Alsace,...,146978,1852325,2011-12,2.0,2.0,2.0,2011-12,2.0,2.0,2.0
1,5524,201152,72,AQUITAINE,24,2011,12,2011-12-01,AQUITAINE,Aquitaine,...,344875,3254233,2011-12,2.0,2.0,2.0,2011-12,2.0,2.0,2.0
2,5525,201152,83,AUVERGNE,91,2011,12,2011-12-01,AUVERGNE,Auvergne,...,150880,1350682,2011-12,3.0,3.0,2.0,2011-12,3.0,3.0,2.0
3,5526,201152,25,BASSE-NORMANDIE,49,2011,12,2011-12-01,BASSENORMANDIE,Basse-Normandie,...,154315,1475684,2011-12,2.0,2.0,2.0,2011-12,2.0,2.0,2.0
4,5527,201152,26,BOURGOGNE,33,2011,12,2011-12-01,BOURGOGNE,Bourgogne,...,181201,1642734,2011-12,2.0,2.0,1.0,2011-12,2.0,2.0,1.0


In [9]:
train_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           9196 non-null   int64  
 1   week         9196 non-null   int64  
 2   region_code  9196 non-null   int64  
 3   region_name  9196 non-null   object 
 4   TauxGrippe   9196 non-null   int64  
 5   year         9196 non-null   int64  
 6   week_num     9196 non-null   int64  
 7   date         9196 non-null   object 
 8   month        9196 non-null   int64  
 9   t            9196 non-null   float64
 10  pmer         9196 non-null   float64
 11  ff           9196 non-null   float64
 12  rr24         9196 non-null   float64
 13  u            9196 non-null   float64
dtypes: float64(5), int64(7), object(2)
memory usage: 1005.9+ KB


In [10]:
train_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 24 columns):
 #   Column                                                                                Non-Null Count  Dtype  
---  ------                                                                                --------------  -----  
 0   Id                                                                                    9196 non-null   int64  
 1   week                                                                                  9196 non-null   int64  
 2   region_code                                                                           9196 non-null   int64  
 3   region_name                                                                           9196 non-null   object 
 4   TauxGrippe                                                                            9196 non-null   int64  
 5   year                                                                               

## Séléction des colonnes

In [12]:
cols_meteo = [
    "Id",
    "week",
    "region_code",
    "region_name",
    "TauxGrippe",
    "year",
    "week_num",
    "month",
    "t",
    "pmer",
    "ff",
    "rr24",
    "u"
]

train_meteo = train_meteo[cols_meteo]

In [13]:
cols_google = [
    "Id",
    "week",
    "region_code",
    # variables Google
    "requete_grippe_x",
    "requete_grippe_aviaire_vaccin_x",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_x",
    "requete_grippe_y",
    "requete_grippe_aviaire_vaccin_y",
    "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_y"
]

train_google = train_google[cols_google]

In [14]:
assert not train_meteo.duplicated(["region_code", "week"]).any()
assert not train_google.duplicated(["region_code", "week"]).any()

In [15]:
train_full = train_meteo.merge(
    train_google,
    on=["region_code", "week"],
    how="inner",
    suffixes=("", "_google")
)

print(train_full.shape)

(9196, 20)


In [16]:
train_full.head()

,Id,week,region_code,region_name,TauxGrippe,year,week_num,month,t,pmer,ff,rr24,u,Id_google,requete_grippe_x,requete_grippe_aviaire_vaccin_x,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_x,requete_grippe_y,requete_grippe_aviaire_vaccin_y,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie_y
0,5523,201152,42,ALSACE,66,2011,52,12,278.613306,101860.544355,3.887702,179.4,83.187500,5523,2.0,2.0,2.0,2.0,2.0,2.0
1,5524,201152,72,AQUITAINE,24,2011,52,12,282.006707,102447.682927,3.197154,305.3,87.701220,5524,2.0,2.0,2.0,2.0,2.0,2.0
2,5525,201152,83,AUVERGNE,91,2011,52,12,278.906452,102179.798387,3.413911,106.7,80.453629,5525,3.0,3.0,2.0,3.0,3.0,2.0
3,5526,201152,25,BASSE-NORMANDIE,49,2011,52,12,281.327209,101840.124031,6.089130,336.3,85.221705,5526,2.0,2.0,2.0,2.0,2.0,2.0
4,5527,201152,26,BOURGOGNE,33,2011,52,12,278.612857,102035.428571,4.084898,107.2,88.216327,5527,2.0,2.0,1.0,2.0,2.0,1.0


In [17]:
train_full.to_csv("train_full.csv", index=False)